In [1]:
print("Ok")

Ok


In [8]:
%pwd

'd:\\RAG_Medical_Bot_Practice'

In [7]:
import os
os.chdir('../')

In [9]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
# Extract The Data from PDF
def load_pdf_file(data):
    loader = DirectoryLoader(data, 
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [11]:
extracted_data = load_pdf_file(data='Data/')

In [14]:
len(extracted_data)

637

In [12]:
# Split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, 
                                                   chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5860

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
# Download Embedding model

def download_embed_model():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-miniLM-L6-v2')
    return embeddings

In [18]:
embeddings = download_embed_model()

c:\Users\DELL\anaconda3\envs\env-medical_rag_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
pinecone_api_key = os.environ.get('pinecone_api_key')

In [21]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key=pinecone_api_key)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension = 384, # Replace with your model dimensions
    metric = "cosine", # Replace with your model metric
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    ) 
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-ko283zj.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings
)

In [23]:
# Existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding=embeddings
)


In [25]:
retriever = docsearch.as_retriever(search_type = "similarity",
                       search_kwargs = {"k":3})

In [26]:
ret_docs = retriever.invoke("what is Acne?")

In [27]:
ret_docs

[Document(id='f10b130f-35e7-4256-a9ae-aacb676fb043', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='10ca132d-3a89-48c8-a3b5-d7ca7aac1692', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [29]:
from langchain_groq import ChatGroq

In [44]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.4,
    max_tokens=50
)

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [46]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [47]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [48]:
qa_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

In [52]:
response = rag_chain.invoke({"input":"What is Gigatism?"})

In [53]:
response['answer']

'Gigantism is a variant of acromegaly that occurs in children whose bony growth plates have not closed, resulting in exceptional growth of long bones due to the abnormal release of growth hormone from the pituitary gland. This condition causes'